In [195]:
import pandas as pd
import sqlite3


In [196]:
def return_table(begin_date2, end_date2):
    # Set up sqlite
    connection = sqlite3.connect('../static/data/sensors_readings_2016_present.db')
    
    # Assemble Query with proper placement of WHERE clause and using parameterized queries
    sql_query = """
    SELECT sensor_id, latitude, longitude, altitude, AVG(pm2) AS avg_pm2, AVG(pm10) AS avg_pm10
    FROM sensors_readings
    WHERE date BETWEEN ? AND ?
    GROUP BY sensor_id, latitude, longitude, altitude
    """
    
    # Execute the query with parameter substitution to prevent SQL injection
    df = pd.read_sql_query(sql_query, connection, params=(begin_date2, end_date2))
    
    connection.close()
    return df

In [197]:
df = return_table('2023-09-27','2023-09-30')


In [198]:
df_color = pd.read_csv('../static/data/sensor_categories.csv')
df = pd.merge(df,df_color, on = 'sensor_id')

In [199]:
averages = df.groupby('category').max().reset_index()
averages = averages[['category','avg_pm2','avg_pm10']]
averages.rename(columns = {'avg_pm2':'cat_avg_pm2','avg_pm10':'cat_avg_pm10'}, inplace = True)
averages['cat_avg_pm2'] = round(averages['cat_avg_pm2']).astype('int')
averages['cat_avg_pm10'] = round(averages['cat_avg_pm10']).astype('int')

In [200]:
df['avg_pm2'] = round(df['avg_pm2']).astype('int')
df['avg_pm10'] = round(df['avg_pm10']).astype('int')

df = pd.merge(df,averages, on = 'category')
df

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,category,cat_avg_pm2,cat_avg_pm10
0,77,40.750816,-111.825290,NaN,18,18,red,3323,3323
1,443,41.224422,-111.968376,NaN,7,7,red,3323,3323
2,525,40.783870,-111.870140,4891.0,1,1,yellow,1773,1774
3,984,40.595387,-111.807755,NaN,2,2,green,1667,1667
4,3238,40.664246,-111.849230,4331.0,3,3,green,1667,1667
...,...,...,...,...,...,...,...,...,...
195,169779,40.769947,-111.847015,4758.0,2,3,red,3323,3323
196,169851,40.902760,-111.872420,4361.0,2,3,orange,1664,1664
197,169867,40.871600,-111.903200,4306.0,2,4,green,1667,1667
198,174555,40.722404,-111.823616,4621.0,2,2,green,1667,1667


In [201]:
df.cat_avg_pm2.unique()

array([3323, 1773, 1667, 1664])

In [150]:
df_color

,sensor_id,category
0,77,red
1,443,red
2,525,yellow
3,984,green
4,992,yellow
...,...,...
217,204009,red
218,204315,yellow
219,207743,green
220,208781,yellow


In [56]:
# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')

# Now you can sort by this column
df_sorted = df.sort_values(by='date')

In [60]:
df_sorted.groupby('date').count().reset_index()[['date']].to_csv('date_range.csv', index=False)